In [0]:
import os
import sys    
print("Python version: {}".format(sys.version_info[0]))

import warnings
warnings.filterwarnings('ignore') # to suppress unnecessary warnings

try:
    import torch
    print("PyTorch is already installed, good to go!") 
except:
    from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
    platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
    accelerator = 'cu80' if os.path.exists('/opt/bin/nvidia-smi') else 'cpu'
    !pip install -q \
      http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl \
      torchvision
    print("Successfully installed PyTorch!")
    import torch

Python version: 3
PyTorch is already installed, good to go!


In [0]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms

import matplotlib.pyplot as plt  # for plots
import numpy as np  # for working with numbers and arrays of numbers

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

import random
seed = 1
random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)
print("Setup is done!")

Setup is done!


In [0]:
mnist_dir = 'datasets/MNIST' # the directory our data will be downloaded too

transformed_dataset = datasets.MNIST(mnist_dir, train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,)) # normalizing to the [-1,1] range
                       ]))

for i in range(len(transformed_dataset)):
    sample = transformed_dataset[i]

    print(i, sample[0].size(), sample[1].size())

    if i == 3:
        break

0 torch.Size([1, 28, 28]) torch.Size([])
1 torch.Size([1, 28, 28]) torch.Size([])
2 torch.Size([1, 28, 28]) torch.Size([])
3 torch.Size([1, 28, 28]) torch.Size([])


In [0]:
def load_MNIST(mnist_dir, batch_size, test_batch_size, kwargs):

  # Dataloader object is used to organize training data neatly into batches
  # it is a part of the PyTorch framework
  train_loader = torch.utils.data.DataLoader(
        datasets.MNIST(mnist_dir, train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=batch_size, shuffle=True, **kwargs)
  test_loader = torch.utils.data.DataLoader(
        datasets.MNIST(mnist_dir, train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=test_batch_size, shuffle=True, **kwargs)
  
  return train_loader, test_loader


batch_size = 50 
test_batch_size = 100
mnist_dir = 'datasets/MNIST' # the directory our data will be downloaded too
train_loader, test_loader = load_MNIST(mnist_dir, batch_size, test_batch_size, kwargs)
print("The MNIST data is loaded successfully!")

The MNIST data is loaded successfully!


In [0]:
class CNN(nn.Module):
    def __init__(self):
        ########################################################################
        # modify code here:
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5)  # first convolutional layer
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5) # second convolutional layer
        self.fc1 = nn.Linear(1024, 128) # fully connected layer 1
        self.fc2 = nn.Linear(128, 10) # fully connected layer 2
        ########################################################################

    def forward(self, x):
        ########################################################################
        # modify code here:
        x = self.conv1(x)
        x = F.max_pool2d(x, 2)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.max_pool2d(x, 2)
        x = F.relu(x)
        x = x.view(-1, 1024)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        ########################################################################
        return F.log_softmax(x, dim=1)
    
cnn_model_2 = CNN().to(device)    

In [0]:
def train(model, device, train_loader, loss_function, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        # data has a batch of images
        # target is a batch of corresponding training labels
        optimizer.zero_grad() # initialize all the gradients with zeros
        output = model(data) # compute the network's predictions for the batch of images
        loss = loss_function(output, target) # here we calculate the loss value
        loss.backward() # compute the gradients for all the weights of the network automatically!
        optimizer.step() # here we actually update the network's weights
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))         
print('Training method loaded!')

Training method loaded!


In [0]:
loss_function = nn.NLLLoss()
optimizer = optim.SGD(cnn_model_2.parameters(), lr=0.01, momentum=0.9) # lr - learning rate
num_epochs = 5
for epoch in range(1, num_epochs + 1):
        train(cnn_model_2, device, train_loader, loss_function, optimizer, epoch)
print('Training finished') 

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.347785
Train Epoch: 1 [5000/60000 (8%)]	Loss: 0.402990
Train Epoch: 1 [10000/60000 (17%)]	Loss: 0.625150
Train Epoch: 1 [15000/60000 (25%)]	Loss: 0.120014
Train Epoch: 1 [20000/60000 (33%)]	Loss: 0.192006
Train Epoch: 1 [25000/60000 (42%)]	Loss: 0.092286
Train Epoch: 1 [30000/60000 (50%)]	Loss: 0.153471
Train Epoch: 1 [35000/60000 (58%)]	Loss: 0.038535
Train Epoch: 1 [40000/60000 (67%)]	Loss: 0.267275
Train Epoch: 1 [45000/60000 (75%)]	Loss: 0.038398
Train Epoch: 1 [50000/60000 (83%)]	Loss: 0.157141
Train Epoch: 1 [55000/60000 (92%)]	Loss: 0.041399
Train Epoch: 2 [0/60000 (0%)]	Loss: 0.088380
Train Epoch: 2 [5000/60000 (8%)]	Loss: 0.220250
Train Epoch: 2 [10000/60000 (17%)]	Loss: 0.102368
Train Epoch: 2 [15000/60000 (25%)]	Loss: 0.026434
Train Epoch: 2 [20000/60000 (33%)]	Loss: 0.007311
Train Epoch: 2 [25000/60000 (42%)]	Loss: 0.011885
Train Epoch: 2 [30000/60000 (50%)]	Loss: 0.064603
Train Epoch: 2 [35000/60000 (58%)]	Loss: 0.098019
Train Epoch: 2

In [0]:
def test(model, device, test_loader):
    model.eval()
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(test_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += (predicted == targets).sum().item()
        if batch_idx % 10 == 0:
            print('[{}/{}]\t'
                  'Correct: {}\t'
                  'Total: {}\t'.format(
                batch_idx,
                len(test_loader),
                correct,
                total))
    if len(test_loader.dataset)==0:
        print("Error: test data have not been loaded correctly")
        return -1
    
    accuracy = 100.0 * correct / total

    print('Accuracy of the network on the test images: {:.3f}%; correct: {} out of {}'.format(
        accuracy, correct, total))
    
print("Test method is loaded successfully")    

Test method is loaded successfully


In [0]:
test(cnn_model_2, device, test_loader)

[0/100]	Correct: 99	Total: 100	
[10/100]	Correct: 1095	Total: 1100	
[20/100]	Correct: 2088	Total: 2100	
[30/100]	Correct: 3081	Total: 3100	
[40/100]	Correct: 4070	Total: 4100	
[50/100]	Correct: 5062	Total: 5100	
[60/100]	Correct: 6055	Total: 6100	
[70/100]	Correct: 7047	Total: 7100	
[80/100]	Correct: 8035	Total: 8100	
[90/100]	Correct: 9025	Total: 9100	
Accuracy of the network on the test images: 99.200%; correct: 9920 out of 10000
